# Analysis for Performance and Energy Consumption

## Extract information

In [4]:
# Libraries

import sys
import os
import glob
import webbrowser
import natsort
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MaxAbsScaler
from IPython.display import display, HTML


In [5]:
# Directory
actual_directory = os.getcwd() + '/'  

python_releaseDates = {
    'Python 3.13.0a0': '2023-06-07',
    'Python 3.12.0b1': '2023-05-22',
    'Python 3.11.3': '2023-04-05',
    'Python 3.10.11': '2023-04-05',
    'Python 3.9.16': '2022-12-06',
    'Python 3.8.16': '2022-12-06',
    'Python 3.7.16': '2022-12-06',
    'Python 3.6.15': '2021-09-04',
    'Python 3.5.10': '2020-09-05',
    'Python 3.4.10': '2019-03-18',
    'Python 3.3.7': '2017-09-19',
    'Python 3.2.6': '2014-10-11',
    'Python 3.1.5': '2012-04-09',
    'Python 3.0.1': '2009-02-13',
    'Python 2.7.18': '2020-04-20',
    'Python 2.6.9': '2013-10-29',
    'Python 2.5.6': '2011-05-26',
}

cplusplus_releaseDates = {
    'g++-4.4 4.4.7': '2012-03-13',
    'g++-4.6 4.6.4': '2013-04-12',
    'g++-4.7 4.7.4': '2014-06-12',
    'g++-4.8 4.8.5': '2015-06-23',
    'g++-4.9 4.9.3': '2015-06-26',
    'g++-5 5.5.0': '2017-10-10',
    'g++-6 6.5.0': '2018-10-26',
    'g++-7 7.5.0': '2019-11-14',
    'g++-8 8.5.0': '2021-05-14',
    'g++-8 ': '2021-05-14',
    'g++-9 9.5.0': '2022-05-27',
    'g++-10 10.4.0': '2022-06-28',
    'g++-10 10.5.0': '2023-07-07',
    'g++-11 11.4.0': '2023-05-29',
    'g++-12 12.3.0': '2023-05-08',
    'g++-13 13.1.0': '2023-04-26',
}

java_releaseDates = {
    '1.8.0_362': '2023-04-18',
    '9.0.4': '2018-01-16',
    '10.0.2': '2018-07-17',
    '11.0.19': '2020-10-20',
    '12.0.2': '2019-07-16',
    '13.0.2': '2020-01-14',
    '14.0.2': '2020-07-14',
    '15.0.2': '2021-01-19',
    '16.0.2': '2021-07-20',
    '17.0.7': '2023-04-18',
    '18.0.2-ea': '2022-07-19',
    '19.0.2': '2023-01-17',
    '20.0.2': '2023-07-18',
}

js_releaseDates = {
    '20.5.1': '2023-08-09',
    '19.9.0': '2023-04-10',
    '18.17.1': '2023-08-08',
    '17.9.1': '2022-06-01',
    '16.20.2': '2023-08-08',
    '15.14.0': '2021-04-06',
    '14.21.3': '2023-02-16',
    '13.14.0': '2020-04-29',
    '12.22.12': '2022-04-05',
    '11.15.0': '2019-04-30',
    '10.24.1': '2021-04-06',
    '9.11.2': '2018-06-12',
    '8.17.0': '2019-12-17',
    '7.10.1': '2017-07-11',
    '6.17.1': '2019-04-03',
    '5.12.0': '2016-06-23',
    '4.9.1': '2018-03-29',
    '3.3.1': '2015-09-15',
    '2.5.0': '2015-07-28',
    '1.8.4': '2015-07-09',
    '0.12.18': '2017-02-22',
    '0.10.48': '2016-10-18',
    '0.8.28': '2014-07-31'
}

# Function to get release date for a given Python version
def get_release_date(version):
    if language == 'python':
        return python_releaseDates.get(version, 'Unknown')
    elif language == 'c++':
        return cplusplus_releaseDates.get(version, 'Unknown')
    elif language == 'java':
        return java_releaseDates.get(version, 'Unknown')
    elif language == 'js':
        return js_releaseDates.get(version, 'Unknown')

# Function to convert "g", "m" or KiB to "byte"
def convert_g_to_byte(value):
    value_str = str(value)
    if value_str[:-1] == "":
        Byte_value = 0
    elif value_str[-1].lower() == 'g':
        g_value = float(value_str[:-1])
        Byte_value = g_value * 1024 * 1024 * 1024  # 1 giga = 1,000,000,000
    elif value_str[-1].lower() == 'm':
        m_value = float(value_str[:-1])
        Byte_value = m_value * 1024 * 1024 # 1 giga = 1,000,000
    elif value_str[-1].lower() == 'k':
        k_value = float(value_str[:-1])
        Byte_value = k_value * 1024 # 1 kilo = 1,000
    else:
        k_value = float(value_str)
        Byte_value = k_value * 1024 # 1 kilo = 1,000
    return int(Byte_value)

def convert_toUnit(column):
    column = column.apply(convert_g_to_byte)
    column = pd.to_numeric(column, errors='coerce')
    return column
    
def Data_normalized(df, tool):
    df_data = df[['version', 'release_date', 'path', 'appplication']]
    if tool == "turbostat": 
        df_metric = df.loc[:, ~df.columns.isin(['version', 'release_date', 'path', 'appplication'])]
    elif tool == "top":
        df_metric = df[['virt', 'res', 'shr', 'percent_cpu', 'percent_mem',
                    'nTH', 'P', 'SWAP', 'CODE', 'DATA', 'nMaj',
                    'nDRT', 'USED']]

    transformer = MaxAbsScaler().fit(df_metric)
    scaled = transformer.transform(df_metric)

    df_norm = pd.DataFrame(scaled, columns=df_metric.columns)
    df = pd.concat([df_data,df_norm.reindex(df_data.index)], axis=1)
    return df

# Function to extract information in every file
def from_CSVfile(file, directory, tool):
     # Read CSV file
    df = pd.read_csv(file)
    if language == 'js': df['version'] = df['version'].str.replace('v', '')
    
    # New column 'release_date' as the second 
    df['release_date'] = df['version'].apply(get_release_date)
    df.insert(1, 'release_date', df.pop('release_date'))

    # Convert date into datetime
    df['release_date'] = pd.to_datetime(df['release_date'])

    # New column 'path' as the third
    df['path'] = directory
    df.insert(2, 'path', df.pop('path'))

    # Clean and remove the unnecessary rows
    df.replace(to_replace='-', value=0, inplace=True)

    # Apply the conversion function to the DataFrame column
    if tool == "top":
        df = df.dropna(subset=['command'])
        df['virt'] = convert_toUnit(df['virt'])
        df['res'] = convert_toUnit(df['res'])
        df['shr'] = convert_toUnit(df['shr'])
        df['CODE'] = convert_toUnit(df['CODE'])
        df['DATA'] = convert_toUnit(df['DATA'])
        df['SWAP'] = convert_toUnit(df['SWAP'])
        df['USED'] = convert_toUnit(df['USED'])
        df['nMin'] = convert_toUnit(df['nMin'])
        df['nMaj'] = convert_toUnit(df['nMaj'])

    # Changes in the 'version' column
    if language == 'python': df['version'] = df['version'].str.replace('Python ', '')
    if language == 'c++': df['version'] = df['version'].str.split().str[0]

    # with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    #     print(df)

    return df

# Extract all information from ALL files
def from_CSVfiles(tool, norm):

    all_df = pd.DataFrame()

    list_directories = next(os.walk(language))[1]
    list_directories.sort()

    for directory_name in list_directories:
        # if directory_name != "older" and directory_name != "test" and directory_name != "general_plots" and directory_name != "general_plots_v1":
        
        if directory_name == "binaryTrees_21_original":

            path = language + '/' + directory_name + '/' + tool + '/'
            if tool == "perf": path = path + tool + extra

            if tool == "top":

                df = pd.DataFrame()
                list_files = os.listdir(path)

                # Get list of all files only in the given directory
                list_files = natsort.natsorted(list_files)

                for file_name in list_files:
                    if file_name.startswith("temp_top_data_") and file_name.endswith('.csv'):
                        df_Top = from_CSVfile(path + file_name, directory_name, tool)
                        df = pd.concat([df, df_Top])

                df.to_csv(path + "top_data_allVersions.csv", index=False)
            elif tool == "turbostat":

                df = pd.DataFrame()
                list_files = os.listdir(path)

                # Get list of all files only in the given directory
                list_files = natsort.natsorted(list_files)

                for file_name in list_files:
                    if file_name.startswith("turbostat_performance_data") and file_name.endswith('.csv') and file_name != "turbostat_performance_data_allVersions.csv":
                        df_Top = from_CSVfile(path + file_name, directory_name, tool)
                        df = pd.concat([df, df_Top])

            else:
                df = from_CSVfile(path + '_data_allVersions.csv', directory_name, tool)
                if norm: df = Data_normalized(df, tool)
            all_df = pd.concat([all_df, df])

    return all_df

In [6]:
language = "c++"

df = from_CSVfiles("turbostat", norm=False)
df

test release_date                     path  version        appplication  \
0      1   2012-03-13  binaryTrees_21_original  g++-4.4  binaryTrees_2.c 21   
1      2   2012-03-13  binaryTrees_21_original  g++-4.4  binaryTrees_2.c 21   
2      3   2012-03-13  binaryTrees_21_original  g++-4.4  binaryTrees_2.c 21   
3      4   2012-03-13  binaryTrees_21_original  g++-4.4  binaryTrees_2.c 21   
4      5   2012-03-13  binaryTrees_21_original  g++-4.4  binaryTrees_2.c 21   
..   ...          ...                      ...      ...                 ...   
5      6   2023-04-26  binaryTrees_21_original   g++-13  binaryTrees_2.c 21   
6      7   2023-04-26  binaryTrees_21_original   g++-13  binaryTrees_2.c 21   
7      8   2023-04-26  binaryTrees_21_original   g++-13  binaryTrees_2.c 21   
8      9   2023-04-26  binaryTrees_21_original   g++-13  binaryTrees_2.c 21   
9     10   2023-04-26  binaryTrees_21_original   g++-13  binaryTrees_2.c 21   

    time_elapsed  usec  Time_Of_Day_Seconds  APIC  X2APIC  ...  Pkg%pc9  \
0      19.012345  1575         1.692782e+09     0       0  ...      0.0   
1      19.068004  1643         1.692782e+09     0       0  ...      0.0   
2      19.485950  1584         1.692782e+09     0       0  ...      0.0   
3      19.485471  1601         1.692782e+09     0       0  ...      0.0   
4      19.006089  1641         1.692782e+09     0       0  ...      0.0   
..           ...   ...                  ...   ...     ...  ...      ...   
5      20.287041  1659         1.692786e+09     0       0  ...      0.0   
6      19.587371  1647         1.692786e+09     0       0  ...      0.0   
7      20.397967  1678         1.692786e+09     0       0  ...      0.0   
8      19.797566  1581         1.692786e+09     0       0  ...      0.0   
9      20.948406  1589         1.692786e+09     0       0  ...      0.0   

    Pk%pc10  CPU%LPI  SYS%LPI   Pkg_J   Cor_J  GFX_J  RAM_J  PKG_%  RAM_%  
0       0.0      0.0      0.0  276.73  228.10   0.26  14.03    0.0    0.0  
1       0.0      0.0      0.0  283.23  234.26   0.28  14.00    0.0    0.0  
2       0.0      0.0      0.0  292.54  242.32   0.27  14.19    0.0    0.0  
3       0.0      0.0      0.0  293.64  243.25   0.28  14.27    0.0    0.0  
4       0.0      0.0      0.0  284.07  234.74   0.27  13.97    0.0    0.0  
..      ...      ...      ...     ...     ...    ...    ...    ...    ...  
5       0.0      0.0      0.0  309.72  256.90   0.28  14.76    0.0    0.0  
6       0.0      0.0      0.0  296.12  245.20   0.29  14.47    0.0    0.0  
7       0.0      0.0      0.0  293.94  240.95   0.28  14.87    0.0    0.0  
8       0.0      0.0      0.0  291.21  239.59   0.28  14.61    0.0    0.0  
9       0.0      0.0      0.0  264.71  210.28   0.28  15.17    0.0    0.0  

[140 rows x 63 columns]

In [65]:
df_median = df.groupby(["version","path"], sort=False)[["Pkg_J","Cor_J","RAM_J", "time_elapsed"]].median().reset_index()
df_median

version                     path    Pkg_J    Cor_J   RAM_J  time_elapsed
0   g++-4.4  binaryTrees_21_original  291.800  241.565  14.165     19.269851
1   g++-4.6  binaryTrees_21_original  277.860  229.085  13.790     18.632323
2   g++-4.7  binaryTrees_21_original  279.755  231.575  13.855     18.763978
3   g++-4.8  binaryTrees_21_original  280.020  231.135  13.845     18.770183
4   g++-4.9  binaryTrees_21_original  275.015  226.885  13.740     18.530828
5     g++-5  binaryTrees_21_original  271.960  223.825  13.710     18.495283
6     g++-6  binaryTrees_21_original  275.320  225.695  13.895     18.846100
7     g++-7  binaryTrees_21_original  283.110  232.200  14.065     19.115922
8     g++-8  binaryTrees_21_original  283.805  234.205  14.040     19.102328
9     g++-9  binaryTrees_21_original  289.715  238.970  14.340     19.674847
10   g++-10  binaryTrees_21_original  288.445  237.465  14.255     19.560124
11   g++-11  binaryTrees_21_original  294.060  243.140  14.390     19.722264
12   g++-12  binaryTrees_21_original  292.690  241.770  14.565     19.864070
13   g++-13  binaryTrees_21_original  294.590  243.200  14.495     19.771586

In [132]:
import statsmodels.api as sm

fig = px.scatter(df, x='time_elapsed', y='Pkg_J', color='version', trendline="ols", trendline_scope="overall")
fig.show()

In [106]:
fig = px.scatter(df, x='time_elapsed', y='Pkg_J', color='version', marginal_x="box", marginal_y="box", height=1500, width=1500,)
fig.show()

In [24]:
import plotly.figure_factory as ff

In [66]:
fig = ff.create_scatterplotmatrix(df[['time_elapsed', 'Pkg_J', 'RAM_J', 'version']], 
                                  height=1000,
                                  width=1000,
                                  diag='histogram',
                                  index='version')
fig.show()

In [67]:
fig = ff.create_scatterplotmatrix(df_median[['time_elapsed', 'Pkg_J', 'RAM_J', 'version']], 
                                  height=1000,
                                  width=1000,
                                  diag='histogram',
                                  index='version')
fig.show()

In [93]:
df[['time_elapsed','Avg_MHz', 'Busy%', 'IPC', 'IRQ', 'POLL', 'C1%','C1E%','C3%','C6%','C7s%','C8%','C9%','C10%','CPU%c1','CPU%c3','CPU%c6','CPU%c7','CoreTmp','PkgTmp','GFX%rc6','GFXMHz','GFXAMHz','Totl%C0','Any%C0','GFX%C0','CPUGFX%']].corr()

time_elapsed   Avg_MHz     Busy%       IPC       IRQ      POLL  \
time_elapsed      1.000000  0.068107  0.192450 -0.596731  0.456779  0.219338   
Avg_MHz           0.068107  1.000000  0.949295 -0.639685  0.127118  0.664125   
Busy%             0.192450  0.949295  1.000000 -0.649045  0.275152  0.656393   
IPC              -0.596731 -0.639685 -0.649045  1.000000 -0.301365 -0.530760   
IRQ               0.456779  0.127118  0.275152 -0.301365  1.000000  0.182587   
POLL              0.219338  0.664125  0.656393 -0.530760  0.182587  1.000000   
C1%               0.193786  0.843616  0.909427 -0.634232  0.230824  0.570681   
C1E%              0.214358  0.896732  0.923712 -0.563347  0.279776  0.681866   
C3%               0.209081  0.863040  0.891452 -0.531579  0.359766  0.649342   
C6%               0.334349  0.562058  0.663989 -0.442463  0.795369  0.509147   
C7s%              0.021084  0.297209  0.279340 -0.222202  0.070062  0.449100   
C8%               0.334464  0.313326  0.471300 -0.336851  0.879418  0.346791   
C9%               0.379137 -0.085636  0.066012 -0.137674  0.915503 -0.002995   
C10%             -0.399859 -0.296402 -0.455515  0.363605 -0.942666 -0.289056   
CPU%c1            0.317237  0.811367  0.916425 -0.617030  0.604021  0.597984   
CPU%c3            0.197873  0.819194  0.843325 -0.516490  0.331619  0.631175   
CPU%c6            0.344543  0.190644  0.316256 -0.266207  0.902556  0.257211   
CPU%c7           -0.293323 -0.854500 -0.946757  0.633362 -0.555162 -0.628899   
CoreTmp           0.470129 -0.081253  0.082701 -0.127857  0.479205  0.015621   
PkgTmp            0.502612 -0.066351  0.101449 -0.151094  0.536046  0.049007   
GFX%rc6           0.169322  0.214902  0.227728 -0.190811  0.100105  0.213706   
GFXMHz            0.247453 -0.068422 -0.025784 -0.069360 -0.027448 -0.032209   
GFXAMHz                NaN       NaN       NaN       NaN       NaN       NaN   
Totl%C0           0.195840  0.943248  0.998492 -0.645058  0.291727  0.651839   
Any%C0           -0.380930  0.135033 -0.016913  0.102184 -0.943534  0.009984   
GFX%C0           -0.435948 -0.004125 -0.055614  0.240681 -0.093802 -0.095825   
CPUGFX%          -0.472884  0.011713 -0.043858  0.251673 -0.147461 -0.098435   

                   C1%      C1E%       C3%       C6%  ...    CPU%c7   CoreTmp  \
time_elapsed  0.193786  0.214358  0.209081  0.334349  ... -0.293323  0.470129   
Avg_MHz       0.843616  0.896732  0.863040  0.562058  ... -0.854500 -0.081253   
Busy%         0.909427  0.923712  0.891452  0.663989  ... -0.946757  0.082701   
IPC          -0.634232 -0.563347 -0.531579 -0.442463  ...  0.633362 -0.127857   
IRQ           0.230824  0.279776  0.359766  0.795369  ... -0.555162  0.479205   
POLL          0.570681  0.681866  0.649342  0.509147  ... -0.628899  0.015621   
C1%           1.000000  0.790677  0.717365  0.509290  ... -0.858664  0.018379   
C1E%          0.790677  1.000000  0.933318  0.687771  ... -0.895891  0.066967   
C3%           0.717365  0.933318  1.000000  0.752257  ... -0.884374  0.134988   
C6%           0.509290  0.687771  0.752257  1.000000  ... -0.841668  0.340964   
C7s%          0.182891  0.305797  0.317717  0.239579  ... -0.264949 -0.024045   
C8%           0.420253  0.478604  0.521689  0.834892  ... -0.691231  0.437115   
C9%           0.057653  0.056682  0.133392  0.621930  ... -0.345720  0.538260   
C10%         -0.407429 -0.439048 -0.494633 -0.850805  ...  0.693484 -0.500906   
CPU%c1        0.861864  0.870589  0.848913  0.830733  ... -0.991468  0.204868   
CPU%c3        0.670682  0.901049  0.967418  0.720822  ... -0.838745  0.088703   
CPU%c6        0.197144  0.337692  0.411210  0.886821  ... -0.585401  0.424773   
CPU%c7       -0.858664 -0.895891 -0.884374 -0.841668  ...  1.000000 -0.201196   
CoreTmp       0.018379  0.066967  0.134988  0.340964  ... -0.201196  1.000000   
PkgTmp        0.031918  0.088624  0.163151  0.395172  ... -0.235959  0.986074   
GFX%rc6       0.306434  0.236198  0.172945  0.136172  ... -0.234938 -0.3

In [95]:
fig = px.parallel_coordinates(df[['time_elapsed','Avg_MHz', 'Busy%', 'IPC', 'IRQ', 'POLL', 'C1%','C1E%','C3%','C6%','C7s%','C8%','C9%','C10%','CPU%c1','CPU%c3','CPU%c6','CPU%c7','CoreTmp','PkgTmp','GFX%rc6','GFXMHz','GFXAMHz','Totl%C0','Any%C0','GFX%C0','CPUGFX%','Pkg_J','Cor_J','GFX_J','RAM_J']],
                              
)

fig.show()

In [96]:
corrs = df[['time_elapsed','Avg_MHz', 'Busy%', 'IPC', 'IRQ', 'POLL', 'C1%','C1E%','C3%','C6%','C7s%','C8%','C9%','C10%','CPU%c1','CPU%c3','CPU%c6','CPU%c7','CoreTmp','PkgTmp','GFX%rc6','GFXMHz','GFXAMHz','Totl%C0','Any%C0','GFX%C0','CPUGFX%','Pkg_J','Cor_J','GFX_J','RAM_J']].corr().round(2) 

figure = ff.create_annotated_heatmap(z=corrs.values,
                                     x=list(corrs.columns),
                                     y=list(corrs.index),
                                     xgap=1, ygap=1,
                                     annotation_text=corrs.round(2).values,
                                     colorscale="Inferno",
                                     showscale=True, reversescale=True)
figure.show()

In [98]:
heat = go.Heatmap(z=corrs.values,
                  x=list(corrs.columns),
                  y=list(corrs.index),
                  xgap=1, ygap=1,
                  texttemplate="%{z}",
                  showscale=True,
                  colorbar_thickness=20,
                  colorbar_ticklen=3,
                  )
layout = go.Layout(title_x=0.5, 
                   width=1500, height=1500,
                   xaxis_showgrid=False,
                   yaxis_showgrid=False,
                   yaxis_autorange='reversed')
fig=go.Figure(data=[heat], layout=layout)        
fig.show() 

In [134]:
px.scatter(df, y="Pkg_J", x="time_elapsed", trendline="ols",
           color="path", animation_frame="version",
           width=1000, height=800,
           range_x=[0,30], range_y=[250,360])

In [139]:
fig = px.ecdf(df, x="Pkg_J", color="version", markers=True, lines=True)
fig.show()

In [ ]:
def general_df():
    df_turbo = from_CSVfiles("turbostat", norm=False)

    df_perf = from_CSVfiles("perf", norm=False)

    df_top = from_CSVfiles("top", norm=False)